In [1]:
import json

file_path = './tokenizer.json'

# Open the JSON file
with open(file_path, 'r') as file:
    # Load the JSON data
    data = json.load(file)

# Print the loaded data (optional)
print(len(data['model']['vocab']))


128000


In [2]:
from datasets import load_dataset

# Load dataset from a local Parquet file

test_dir = [f"dataset/default/partial-train/000{i}.parquet" for i in range(10)]

dataset = load_dataset('parquet', data_files=test_dir)

print(dataset)


/Users/ngkuissi/miniforge3/envs/laion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['__key__', '__url__', 'flac', 'json', 'txt'],
        num_rows: 18900
    })
})


In [3]:
txt = dataset["train"]["txt"]

In [4]:
vocabulary = set()

for sent in dataset["train"]["txt"]:
    for word in sent.split():
        vocabulary.add(word)
len(vocabulary)

4096

In [7]:
stoi = data['model']["vocab"]

In [8]:
itos = {value:key for key,value in stoi.items()}

In [13]:
itos_snac = {int(value):itos[int(value)] for value in vocabulary}

In [ ]:
class Vocabulary:
    """Class to map codes from huggingface dataset to tokens in Llama 3-8B token"""

    def __init__(self):
        self.stoi = {}
        self.itos = {}
    
    def build_vocabulary(self, parquet_files, tokenizer_file="tokenizer.json"):
        '''
        creates the vocabulary from the Llama 3 tokenizer and hugging face dataset
        Args:
            tokenizer_file(str): file downloaded from Llama 3(8B) which contains the vocabulary for the model
            parquet_files(list): director with the dataset from hugging face in parquet format

        '''
        # Open the JSON file
        with open(tokenizer_file, 'r') as file:
            # Load the JSON data
            data = json.load(file)
        
        llama_stoi = data['model']['vocab']
        llama_itos = {value:key for key,value in llama_stoi.items()}

        #load hugging face data
        dataset = load_dataset('parquet', data_files=test_dir)
        vocabulary = set()

        for sent in dataset["train"]["txt"]:
            for word in sent.split():
                vocabulary.add(word)
        
        self.itos = {int(value):llama_itos[int(value)] for value in vocabulary}
        self.stoi = {value:key for key,value in self.itos.items()}
    
    def save(self, file_path):
        with open(file_path, "w") as file:
            json.dump(self.itos, file)
        
        

        